In [1]:
import random


def get_qa(pre_answer=None):
    n1, n2 = random.randint(0, 9999), random.randint(0, 9999)

    q = '%d+%d=' % (n1, n2)
    if pre_answer:
        q = '+%d=' % (n2)
        n1 = int(pre_answer)

    a = '%d' % (n1 + n2)

    return q, a


get_qa(), get_qa('1')

(('1294+3711=', '5005'), ('+124=', '125'))

In [2]:
def get_multi_qa():
    q, a = get_qa()
    text = q

    depth = random.randint(0, 2)

    if depth != 0:
        text += a

    for i in range(depth):
        q, a = get_qa(a)
        text += q
        if i < depth - 1:
            text += a

    return text, a


get_multi_qa()

('6761+646=', '7407')

In [3]:
import random
import torch


class Dataset(torch.utils.data.Dataset):

    def __init__(self, negative_label, with_answer):
        self.negative_label = negative_label
        self.with_answer = with_answer

    def __len__(self):
        return 100_0000

    def __getitem__(self, _):
        q, a = get_multi_qa()

        label = 1
        if self.negative_label and random.random() > 0.5:
            label = 0
            a = int(a)
            a = a + int(random.normalvariate(0, a**0.5)) + random.randint(
                -15, 15)
            if a < 0:
                a = 0
            a = '%d' % a

        if not self.with_answer:
            a = ''

        text = q + a

        return {'text': text, 'label': label}


def get_loader(collate_fn, negative_label, with_answer):
    dataset = Dataset(negative_label=negative_label, with_answer=with_answer)
    return torch.utils.data.DataLoader(dataset,
                                       batch_size=16,
                                       shuffle=True,
                                       drop_last=True,
                                       collate_fn=collate_fn)


next(iter(get_loader(None, negative_label=True, with_answer=True)))

{'text': ['3148+1302=4450+7445=11948',
  '9635+4278=13913',
  '8869+332=9201+1121=10322',
  '7143+1019=8140',
  '5666+8694=14360+1764=16124+432=16556',
  '2757+198=2955+8901=11856+2442=14298',
  '3122+8633=11933',
  '5886+7689=13575',
  '9271+2340=11611',
  '4322+5819=10193',
  '727+4349=5076+4230=9282',
  '1817+4521=6338+3512=9850+3866=13857',
  '7315+9697=17012+8542=25554+9646=35200',
  '9110+4425=13535+7864=21399',
  '377+499=876+2613=3489+8336=11825',
  '907+2202=3109+5324=8435'],
 'label': tensor([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0])}